In [1]:
# May be needed
import os
os.chdir('../mangaki')

In [132]:
from mangaki.utils.data import Dataset

dataset = Dataset()
dataset.load('ratings-svd.pickle')

In [133]:
from scipy.sparse import csc_matrix
import numpy as np

matrix = csc_matrix((dataset.anonymized.y, (dataset.anonymized.X[:, 0], dataset.anonymized.X[:, 1])), dtype=np.float32)

In [134]:
import numpy as np
import tensorflow as tf

X = tf.cast(tf.SparseTensor(
    indices=dataset.anonymized.X,
    values=dataset.anonymized.y,
    dense_shape=(dataset.anonymized.nb_users, dataset.anonymized.nb_works),
), tf.float32)

# ALS

In [135]:
# For evaluation
indices = dataset.anonymized.X

In [136]:
N = dataset.anonymized.nb_users
R = 20
M = dataset.anonymized.nb_works

U = tf.Variable(np.random.rand(N, R).astype(np.float32), name='U')
V = tf.Variable(np.random.rand(R, M).astype(np.float32), name='V')

In [137]:
alpha = tf.constant(1e-4)
regu_U = alpha * tf.reduce_sum(tf.pow(U, 2))
regu_V = alpha * tf.reduce_sum(tf.pow(V, 2))

In [138]:
Y_est = tf.matmul(U, V)
true_loss = tf.reduce_sum(tf.pow(tf.gather_nd(Y_est, indices) - Y_tf.values, 2))
cost = true_loss + regu_U + regu_V

In [139]:
optimizer = tf.train.AdamOptimizer(0.005)
# train_step = optimizer.minimize(cost, var_list=[U, V])
train_U_step = optimizer.minimize(cost, var_list=[U])
train_V_step = optimizer.minimize(cost, var_list=[V])

In [140]:
init_op = tf.global_variables_initializer()
sess.run(init_op)

In [142]:
sess = tf.InteractiveSession()
sess.run(init_op)
for i in range(20):
    sess.run(train_U_step)
    sess.run(train_V_step)
    # sess.run(train_C_step)
    if i % 10 == 0:
        print(i, sess.run(cost), sess.run(true_loss))

0 6.36798e+06 6.36797e+06
10 4.2728e+06 4.27279e+06


- Batch by batch
- Try autoencoder with sigmoid then identity
- Maybe with SparseTensor?

In [143]:
#sess = tf.InteractiveSession()
#print(sess.run((tf.shape(X), tf.shape(V))))

N = dataset.anonymized.nb_users
R = 2
M = dataset.anonymized.nb_works

V = tf.Variable(tf.random_normal([M, R]), name='V')
mu = tf.Variable(tf.random_normal([R]), name='mu')

W = tf.Variable(tf.random_normal([R, M]), name='W')
b = tf.Variable(tf.random_normal([M]), name='b')

encoder = tf.nn.sigmoid(tf.add(tf.sparse_tensor_dense_matmul(X, V), mu))
y_pred = tf.add(tf.matmul(encoder, W), b)
Y_true = X
true_loss = tf.reduce_sum(tf.pow(tf.gather_nd(y_pred, indices) - Y_true.values, 2))

In [144]:
alpha = tf.constant(1e-2)
regu_V = alpha * tf.reduce_sum(tf.pow(V, 2))
regu_W = alpha * tf.reduce_sum(tf.pow(W, 2))
cost = true_loss + regu_V + regu_W

In [145]:
#optimizer = tf.train.AdamOptimizer(0.005)
#optimizer = tf.train.AdamOptimizer(1)
optimizer = tf.train.RMSPropOptimizer(0.1)
train_step = optimizer.minimize(cost, var_list=[V, mu, W, b])
#train_U_step = optimizer.minimize(cost, var_list=[U])
#train_V_step = optimizer.minimize(cost, var_list=[V])

In [146]:
init_op = tf.global_variables_initializer()

In [147]:
sess.run(init_op)
for i in range(20):
    #sess.run(train_U_step)
    #sess.run(train_V_step)
    _, c, t_l = sess.run([train_step, cost, true_loss])
    if i % 1 == 0:
        print(i, c, t_l)

0 1.36879e+06 1.36842e+06
1 934812.0 934460.0
2 834392.0 834044.0
3 771971.0 771625.0
4 727072.0 726728.0
5 693118.0 692776.0
6 665799.0 665458.0
7 644186.0 643847.0
8 626020.0 625683.0
9 610755.0 610419.0
10 597977.0 597642.0
11 587043.0 586709.0
12 577751.0 577418.0
13 569651.0 569318.0
14 563018.0 562687.0
15 556912.0 556581.0
16 551447.0 551117.0
17 546752.0 546422.0
18 542526.0 542197.0
19 538868.0 538540.0


In [148]:
sess.run(tf.gather_nd(y_pred, indices[:5]))

array([ 1.35544467,  0.60000342,  0.66582125,  1.2417649 ,  0.29592541], dtype=float32)

In [149]:
sess.run(Y_true.values[:5])

array([ 2. , -2. ,  0.1,  2. ,  2. ], dtype=float32)